In [25]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [21]:
df = pd.read_csv("/Users/henryfetzer/Documents/ACME/Vol3FallProj/data/Life_Expectancy_Data.csv")

In [22]:
print(df.shape)
df = df.dropna()
df = df.rename(columns={'Life expectancy ': 'Life expectancy', 'under-five deaths ':'under-five deaths',
                        'Measles ':'Measles'})
df.drop(columns=['Country', 'Status'], inplace=True)
print(df.shape)


(2938, 22)
(1649, 20)


In [23]:
# Create target column
df['life_over_70'] = (df['Life expectancy'] >= 70).astype(int)

Index(['Year', 'Life expectancy', 'Adult Mortality', 'infant deaths',
       'Alcohol', 'percentage expenditure', 'Hepatitis B', 'Measles', ' BMI ',
       'under-five deaths', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling'],
      dtype='object')
Year                                 int64
Life expectancy                    float64
Adult Mortality                    float64
infant deaths                        int64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                              int64
 BMI                               float64
under-five deaths                    int64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
GDP      

In [31]:
# Train test split
data = df.drop(columns=['life_over_70', 'Life expectancy', 'Adult Mortality'])
target = df['life_over_70']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

print(X_train.shape)

(1319, 18)


In [32]:
#Grid search for best classifier
# classifier
rfc = RandomForestClassifier()

# specify hyperparameters
param_grid = {"max_depth": [4,8,12],
              "random_state": [43],
              "max_features": ['sqrt', 'log2', None],
              "n_estimators": [100,200]
              }
rfc_gs = GridSearchCV(rfc, param_grid, scoring="f1", n_jobs=-1)

# run the search
rfc_gs.fit(X_train, y_train)


# print best parameters and resulting score
print(rfc_gs.best_params_, rfc_gs.best_score_, sep='\n')

{'max_depth': 12, 'max_features': None, 'n_estimators': 200, 'random_state': 43}
0.9678549374141016


In [34]:
# zip features importance and names together
feature_names = data.columns
features_and_importance = sorted(zip(rfc_gs.best_estimator_.feature_importances_, feature_names), 
                                 reverse=True)

for importance, name in features_and_importance:
    print(f"{name}: {importance}")

Income composition of resources: 0.563502551342364
 thinness 5-9 years: 0.06575186464455186
 HIV/AIDS: 0.053929388725409116
Alcohol: 0.05326696604150363
 thinness  1-19 years: 0.049585035860332784
Schooling: 0.03603780010692025
Total expenditure: 0.0330654533249436
under-five deaths: 0.021749843710642452
 BMI : 0.021599087563967317
infant deaths: 0.016643654341133466
GDP: 0.01664233221579997
percentage expenditure: 0.014693784252457154
Population: 0.009791680900039769
Hepatitis B: 0.009779298377503806
Year: 0.009629433885508805
Polio: 0.00909817818214182
Measles: 0.007791707615000128
Diphtheria : 0.007441938909780173


With just some preliminary data analysis...

Most important: income composition of resources, thinness of 5-9 year olds, and cases of HIV/AIDS

Least important: numer of reported cases for various diseases like Polio